# Defining MDM

In [1]:
def mdm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="RMSE"):
    def error_check():
        rt = 0
        msg = ""
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)

        if (crit != "RMSE" and crit != "MAE" and crit != "MAPE"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    

    error_code = error_check()

    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return

    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    T = float(len(actual_lst))
    
    if crit == "RMSE":
        for actual, p1, p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append((actual - p1) ** 2)
            e2_lst.append((actual - p2) ** 2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    if (crit == "MAE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    if (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
   
    mean_d = pd.Series(d_lst).mean()
    
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    MDM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    MDM_stat = harvey_adj*MDM_stat
    p_value = 2*t.cdf(-abs(MDM_stat), df = T - 1)
    
    dm_return = collections.namedtuple('mdm_return', 'MDM p_value')
    
    rt = dm_return(MDM = MDM_stat, p_value = p_value)
    
    return rt

# Importing real values

In [2]:
import random
import numpy as np
import pandas as pd
from pandas import read_csv
actual_lst = read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,22])
actual_lst=actual_lst.values
actual_lst=actual_lst.flatten()
actual_lst

array([3293.769861, 3304.170012, 3281.370202, 3276.620258, 3226.600001,
       3235.539989, 3238.379832, 3231.120128, 3219.420006, 3209.909838,
       3237.850185, 3254.110192, 3291.600075, 3292.150085, 3329.42974 ,
       3346.169744, 3339.040173, 3329.489861, 3291.640027, 3313.649731,
       3323.94981 , 3333.990012, 3330.449857, 3356.310018, 3377.990027,
       3396.240037, 3362.33026 , 3344.280006, 3364.489961, 3391.760056,
       3386.910039, 3435.870227, 3428.800002, 3417.050037, 3414.310073,
       3403.030216, 3371.919748, 3357.749818, 3325.170072, 3338.629799,
       3348.41982 , 3359.169837, 3354.009906, 3382.750056, 3381.120196,
       3353.860186, 3360.200039, 3354.019991, 3348.350002, 3383.649931,
       3376.090204, 3374.420005, 3419.729881, 3457.210068, 3484.719871,
       3513.129937, 3526.620306, 3544.889711, 3516.990091, 3517.469895,
       3575.590192, 3559.599954, 3533.790217, 3544.260186, 3553.020005,
       3556.439918, 3585.799896, 3585.029959, 3591.019947, 3564.

# Import the predictions of the proposed model

In [3]:
pred1_lst =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,23])
pred1_lst =pred1_lst.values
pred1_lst=pred1_lst.flatten()
pred1_lst

array([3295.9438, 3292.5186, 3286.8171, 3268.233 , 3240.4377, 3232.0999,
       3244.3987, 3240.707 , 3224.8481, 3218.6604, 3234.4863, 3261.4504,
       3279.69  , 3291.1953, 3309.96  , 3335.2827, 3340.331 , 3316.3953,
       3294.5151, 3300.7817, 3321.0996, 3328.3477, 3329.3462, 3339.7559,
       3368.5466, 3386.243 , 3365.971 , 3344.1519, 3359.2834, 3378.6794,
       3391.6533, 3416.074 , 3425.4937, 3412.6355, 3409.302 , 3401.3875,
       3379.7668, 3358.7058, 3334.112 , 3330.7175, 3356.194 , 3358.983 ,
       3352.845 , 3370.276 , 3379.9202, 3361.8145, 3349.2358, 3347.9004,
       3352.2732, 3368.9937, 3375.4138, 3373.9348, 3402.7195, 3440.8313,
       3466.1538, 3490.9302, 3513.5566, 3522.09  , 3509.7522, 3511.9106,
       3545.203 , 3556.4512, 3534.5327, 3525.5044, 3539.81  , 3554.5266,
       3566.479 , 3573.0122, 3573.0996, 3563.4285, 3538.7512, 3497.6123,
       3458.6958, 3460.2026, 3492.1729, 3497.411 , 3488.4836, 3489.047 ,
       3497.0776, 3527.0508, 3599.2783, 3639.6174, 

# BPNN

In [4]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,1])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-15.773101070413638, p_value=4.2659248763442387e-47)
crit：MAE， mdm_return(MDM=-21.569447782725675, p_value=1.054569562912287e-76)
crit：MAPE， mdm_return(MDM=-21.368547658876793, p_value=1.2067875823401608e-75)


# SVR

In [5]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,2])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-18.68304740579593, p_value=1.1646253980815459e-61)
crit：MAE， mdm_return(MDM=-25.904515208062513, p_value=1.1040319455238153e-99)
crit：MAPE， mdm_return(MDM=-25.753078637539325, p_value=7.00974579021806e-99)


# GRU

In [6]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,3])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.137895987624198, p_value=2.6388574047345185e-26)
crit：MAE， mdm_return(MDM=-15.94893409737973, p_value=5.956236605726827e-48)
crit：MAPE， mdm_return(MDM=-15.60300033651743, p_value=2.84139658307207e-46)


# LSTM

In [7]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,4])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-10.194620251232099, p_value=1.3221519805033897e-22)
crit：MAE， mdm_return(MDM=-16.326026712836978, p_value=8.488538769899917e-50)
crit：MAPE， mdm_return(MDM=-15.969440651041753, p_value=4.7316007496345477e-48)


# Transformer

In [8]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,5])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-9.795390370503053, p_value=4.168043625470902e-21)
crit：MAE， mdm_return(MDM=-15.48230552017816, p_value=1.0854919613856615e-45)
crit：MAPE， mdm_return(MDM=-15.355504570279388, p_value=4.417015155919981e-45)


# Informer

In [9]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,6])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.136944079937265, p_value=2.6623012055666015e-26)
crit：MAE， mdm_return(MDM=-17.01631146167964, p_value=3.225303058668589e-53)
crit：MAPE， mdm_return(MDM=-16.616731204069563, p_value=3.1233189737272556e-51)


# TCN

In [10]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,7])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-12.495675565220186, p_value=5.480804562426972e-32)
crit：MAE， mdm_return(MDM=-19.249650510878983, p_value=1.4054903789823423e-64)
crit：MAPE， mdm_return(MDM=-18.91382335607825, p_value=7.587557136655036e-63)


# BiLSTM

In [11]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,8])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-10.42657349974497, p_value=1.7051011481092143e-23)
crit：MAE， mdm_return(MDM=-15.592459182291899, p_value=3.1948096528807863e-46)
crit：MAPE， mdm_return(MDM=-15.294018016103156, p_value=8.707922874435865e-45)


# CNN-BiLSTM-SAM

In [12]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,9])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-9.540389086275463, p_value=3.590167836320791e-20)
crit：MAE， mdm_return(MDM=-15.353188826636972, p_value=4.5314835768550476e-45)
crit：MAPE， mdm_return(MDM=-14.920702621219391, p_value=5.2317357479768365e-43)


# ICEEMDAN-SVR

In [13]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,10])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-20.43001696769819, p_value=1.01915337498768e-70)
crit：MAE， mdm_return(MDM=-23.750620963002152, p_value=2.982518566321458e-88)
crit：MAPE， mdm_return(MDM=-23.719457706141263, p_value=4.3652855474646707e-88)


# CEEMDAN-LSTM

In [14]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,11])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-17.761065478876418, p_value=5.82632724500056e-57)
crit：MAE， mdm_return(MDM=-20.33805631722476, p_value=3.083655548134509e-70)
crit：MAPE， mdm_return(MDM=-20.020173194072793, p_value=1.4064995246122583e-68)


# ICEEMDAN-BiGRU

In [15]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,12])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-17.644375097312423, p_value=2.2658165840857297e-56)
crit：MAE， mdm_return(MDM=-19.424395882250423, p_value=1.7526001656331598e-65)
crit：MAPE， mdm_return(MDM=-18.973735869930472, p_value=3.728993939432442e-63)


# VMD-BiLSTM

In [16]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,13])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-18.477718028119362, p_value=1.3132532281777087e-60)
crit：MAE， mdm_return(MDM=-19.265172939895308, p_value=1.1683908519183672e-64)
crit：MAPE， mdm_return(MDM=-18.740949025230123, p_value=5.874104776098175e-62)


# BiLSTM-SAM-TCN

In [17]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,14])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-9.591928696498586, p_value=2.330842452363377e-20)
crit：MAE， mdm_return(MDM=-14.956296306727914, p_value=3.547228616891275e-43)
crit：MAPE， mdm_return(MDM=-14.639809010576643, p_value=1.106682322627658e-41)


# ICEEMDAN-BiLSTM-SAM-TCN

In [18]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,15])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-7.530806319344125, p_value=1.8770380576435738e-13)
crit：MAE， mdm_return(MDM=-7.376762421902292, p_value=5.464157989311005e-13)
crit：MAPE， mdm_return(MDM=-6.776532164939931, p_value=2.9627835762646045e-11)


# ICEEMDAN-VMD(PSO)-SAM-TCN

In [19]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,16])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-8.128039024695479, p_value=2.5308790237213894e-15)
crit：MAE， mdm_return(MDM=-10.299051128181826, p_value=5.27808468405919e-23)
crit：MAPE， mdm_return(MDM=-10.121919998219271, p_value=2.4961356482566162e-22)


# ICEEMDAN-VMD(PSO)-BiLSTM-SAM

In [20]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,17])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-21.828660609309306, p_value=4.524243498398153e-78)
crit：MAE， mdm_return(MDM=-24.187758066983193, p_value=1.4243984301647004e-90)
crit：MAPE， mdm_return(MDM=-23.488298555154504, p_value=7.3607993301570125e-87)


# ICEEMDAN-VMD(PSO)-BiLSTM-TCN

In [21]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,18])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-25.377025266795016, p_value=6.920621075591848e-97)
crit：MAE， mdm_return(MDM=-30.32124069292927, p_value=7.08506205586702e-123)
crit：MAPE， mdm_return(MDM=-29.50080267548149, p_value=1.3065615158870016e-118)


# CEEMDAN-VMD(PSO)-BiLSTM-SAM-TCN

In [22]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,19])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-8.718838170857632, p_value=2.7905920423509796e-17)
crit：MAE， mdm_return(MDM=-7.270107197794243, p_value=1.1332437916484906e-12)
crit：MAPE， mdm_return(MDM=-7.740515884155358, p_value=4.261259779741307e-14)


# ICEEMDAN-VMD(PSO)-BiLSTM-SAM-CNN

In [23]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,20])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-8.031920924212354, p_value=5.1495347409068065e-15)
crit：MAE， mdm_return(MDM=-6.616307463046803, p_value=8.209293230900402e-11)
crit：MAPE， mdm_return(MDM=-5.877165813479375, p_value=6.948368659982807e-09)


# ICEEMDAN-VMD(PSO)-LSTM-SAM-TCN

In [24]:
pred2_lst  =read_csv('./True value of SSEC Low and all model predictions.csv', index_col=0,usecols=[0,21])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.464360886923034, p_value=1.235303653772783e-27)
crit：MAE， mdm_return(MDM=-11.926058074432413, p_value=1.4820159659054217e-29)
crit：MAPE， mdm_return(MDM=-11.409800415753828, p_value=2.068584032378163e-27)
